In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# -------------------------------
# 1. Load the Dataset and Define Scalers
# -------------------------------
    
# Load main dataset
#Load Datasets
df = pd.read_sas("LLCP2023.XPT", format='xport', encoding="utf-8")
# Select relevant columns
selected_cols = ['_AGE80', '_SEX', '_BMI5', 'EXEROFT1', 'DIABETE4', '_MICHD', 'CHCKDNY2']
df = df[selected_cols].copy()

# Rename columns
rename_dict = {
        '_AGE80': 'age',
        '_SEX': 'gender',
        '_BMI5': 'bmi',
        'EXEROFT1': 'physical_activity',
        'DIABETE4': 'diabetes',
        '_MICHD': 'heart_disease',
        'CHCKDNY2': 'kidney_disease'
    }
df.rename(columns=rename_dict, inplace=True)
    
# Drop missing values
df.dropna(inplace=True)
    
# Normalize `age` and `bmi` using MinMaxScaler
scaler = MinMaxScaler()
df[['age', 'bmi']] = scaler.fit_transform(df[['age', 'bmi']])

# Load meals dataset
data1 = pd.read_csv("detailed_meals_macros_three_disease.csv")

def main(person):
    name = person[4]

    # -------------------------------
    # 2. Load and Normalize Disease Risk Prediction Models
    # -------------------------------
    
    # Load pre-trained disease risk models
    diabetes_model = tf.keras.models.load_model('Diabetes_Risk.h5')
    heart_model = tf.keras.models.load_model('Heart_Disease_Risk.h5')
    kidney_model = tf.keras.models.load_model('Kidney_Disease_Risk.h5')
    
    # Input Data (Example: Male, Lightly Active, Age 45, BMI 40)
    
    age_input = person[0]
    bmi_input = person[1]
    gender = person[2]  
    physical_activity = person[3] 
        
    # Scale input using `scaler`
    scaled_values = scaler.transform(np.array([[age_input, bmi_input]]))
    
    # Prepare scaled input data
    scaled_input_data = np.array([[scaled_values[0][0], gender, scaled_values[0][1], physical_activity]], dtype=np.float32)
    
    # Predict disease risks
    predicted_diabetes_risk = diabetes_model.predict(scaled_input_data)
    predicted_heart_risk = heart_model.predict(scaled_input_data)
    predicted_kidney_risk = kidney_model.predict(scaled_input_data)
    
    # **NO NEED TO INVERSE TRANSFORM THESE RISKS! They are probability scores (0 to 1).**
    print(f"{name} Here is your predicted diseases risks.")
    print("Predicted Diabetes Risk:", predicted_diabetes_risk)
    print("Predicted Heart Disease Risk:", predicted_heart_risk)
    print("Predicted Kidney Disease Risk:", predicted_kidney_risk)
    
    # Extract risk values as scalars
    predicted_heart_risk_value = float(predicted_heart_risk.flatten()[0])
    predicted_diabetes_risk_value = float(predicted_diabetes_risk.flatten()[0])
    predicted_kidney_risk_value = float(predicted_kidney_risk.flatten()[0])
    
    # -------------------------------
    # 3. Normalize Nutritional Data (`Protein`, `Carbohydrates`, `Fat`)
    # -------------------------------
    
        
    # Prepare Inputs and Outputs
    X = data1[['Heart Disease Risk', 'Diabetes Risk', 'Kidney Disease Risk']].values
    y = data1[['Protein', 'Carbohydrates', 'Fat']].values
    
    # Standardize output nutrition data
    scaler2 = StandardScaler()
    y_scaled = scaler2.fit_transform(y)
    
    # **Ensure the risk values are passed correctly to `scaler2.transform()`**
    risk_input = np.array([[predicted_heart_risk_value, predicted_diabetes_risk_value, predicted_kidney_risk_value]])
    
    # Predict nutritional recommendations
    nutrition_recommendation_model = tf.keras.models.load_model('A1.h5', compile=False)
    nutrition_prediction_scaled = nutrition_recommendation_model.predict(risk_input)
    
    # Unscale nutrition prediction
    nutrition_prediction = scaler2.inverse_transform(nutrition_prediction_scaled)
    
    print("Unscaled Nutrition Recommendations:", nutrition_prediction)
    
    # -------------------------------
    # 4. Recommend Recipes Based on Nutrition Needs
    # -------------------------------
    
    # Load recipes dataset
    df_recipes = pd.read_csv("All_Diets.csv")
    
    # Convert nutrition columns to numeric
    df_recipes["Protein(g)"] = pd.to_numeric(df_recipes["Protein(g)"], errors='coerce')
    df_recipes["Carbs(g)"] = pd.to_numeric(df_recipes["Carbs(g)"], errors='coerce')
    df_recipes["Fat(g)"] = pd.to_numeric(df_recipes["Fat(g)"], errors='coerce')
    
    # Drop missing values
    df_recipes.dropna(subset=["Protein(g)", "Carbs(g)", "Fat(g)"], inplace=True)
    
    # Extract target nutritional recommendations
    target_protein = float(nutrition_prediction.flatten()[0])
    target_carbs = float(nutrition_prediction.flatten()[1])
    target_fat = float(nutrition_prediction.flatten()[2])
    
    # Compute Euclidean distance between recipes and recommended nutrition
    df_recipes["distance"] = np.sqrt(
        (df_recipes["Protein(g)"] - target_protein) ** 2 +
        (df_recipes["Carbs(g)"] - target_carbs) ** 2 +
        (df_recipes["Fat(g)"] - target_fat) ** 2
    )
    
    # Sort recipes by distance (closest match first)
    df_sorted = df_recipes.sort_values(by="distance")
    
    # Display the top 10 best-matching recipes
    print(f"{name} Enjoy your week's recipe recommendations")
    print(df_sorted[["Recipe_name", "Protein(g)", "Carbs(g)", "Fat(g)"]].head(10).to_string(index=False))

In [18]:
John = [28, 21, 1, 4, "John is a 28 year old male who is extremely active and has a BMI of 21(acceptable range)."]
#John is a 28 year old male who is extremely active and has a BMI of 21(acceptable range).
Ann = [50, 41, 2, 1, "Ann is a 50 year old female who is very inactive and has a BMI of 41(severe obesity range)."]
#Ann is a 50 year old female who is very inactive and has a BMI of 41(severe obesity range).
Hari = [18, 35, 1, 2, "Hari is a 37 year old male who is moderately active and has a BMI of 30(slightly obese range)."]
#Hari is a 37 year old male who is moderately active and has a BMI of 30(slightly obese range).

people = [John, Ann, Hari]

for p in people:
    main(p)

C:\Users\rsvis\venvs\bhack25\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
John is a 28 year old male who is extremely active and has a BMI of 21(acceptable range). Here is your predicted diseases risks.
Predicted Diabetes Risk: [[0.21121444]]
Predicted Heart Disease Risk: [[0.0559166]]
Predicted Kidney Disease Risk: [[0.00215676]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Unscaled Nutrition Recommendations: [[145.10184 284.97748  74.46936]]
John is a 28 year old male who is extremely active and has a BMI of 21(acceptable range). Enjoy your week's recipe recommendations
                                   Recipe_name  Protein(g)  Carbs(g)  Fat(g)
                      Greek Salad Chicken Pita      133.23    281.65   82.39
                     Mediterranean BBQ Chicken      148.98    296.25   65.78
    Roasted Squash Soup with Turkey Croquettes      146.13    268.78   77.29
Duo of New England and Manhattan Clam Chowders      130.22    279.43   65.20
   

C:\Users\rsvis\venvs\bhack25\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Ann is a 50 year old female who is very inactive and has a BMI of 41(severe obesity range). Here is your predicted diseases risks.
Predicted Diabetes Risk: [[0.5815987]]
Predicted Heart Disease Risk: [[0.01428835]]
Predicted Kidney Disease Risk: [[1.7379861e-07]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


Unscaled Nutrition Recommendations: [[104.299416 221.07321   56.204636]]
Ann is a 50 year old female who is very inactive and has a BMI of 41(severe obesity range). Enjoy your week's recipe recommendations
                        Recipe_name  Protein(g)  Carbs(g)  Fat(g)
     Barbecue Chicken Pizza recipes       91.36    208.40   50.82
          Barbecue Chicken Pizza-ii       91.36    208.40   50.82
               Marla Sarris recipes       97.03    203.40   62.14
     Mediterranean Chickpea Burgers      116.19    238.09   58.18
              Veggie tahini lentils       84.45    227.58   59.23
             Mediterranean Tofu Pie       83.38    214.44   56.91
                    Greek Layer Dip       86.52    207.99   53.49
Mediterranean Mostaccioli with Feta       94.74    204.03   43.24
                   Tunisian Leblebi       98.69    215.10   32.20
   Mediterranean-Style Poached Eggs       81.71    211.34   46.37


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


C:\Users\rsvis\venvs\bhack25\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Hari is a 37 year old male who is moderately active and has a BMI of 30(slightly obese range). Here is your predicted diseases risks.
Predicted Diabetes Risk: [[0.17465301]]
Predicted Heart Disease Risk: [[0.03973944]]
Predicted Kidney Disease Risk: [[0.00012728]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Unscaled Nutrition Recommendations: [[144.69174 282.77216  74.13986]]
Hari is a 37 year old male who is moderately active and has a BMI of 30(slightly obese range). Enjoy your week's recipe recommendations
                                   Recipe_name  Protein(g)  Carbs(g)  Fat(g)
                      Greek Salad Chicken Pita      133.23    281.65   82.39
    Roasted Squash Soup with Turkey Croquettes      146.13    268.78   77.29
                     Mediterranean BBQ Chicken      148.98    296.25   65.78
Duo of New England and Manhattan Clam Chowders      130.22    279.43   65.20
                Keto Beef and 